In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import cartopy, cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.colors as colors
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from PIL import Image
import glob
import warnings
import os
from PIL import Image
import numpy as np
# import rioxarray
import seaborn as sns

In [9]:
def preprocess(f):



    # Define datetime
    date = str(f.date.values[0])
    hours = str(timedelta(seconds=int(f.datesec.values[0]))).split(':')
    year, mon, day = int(date[0:4]), int(date[4:6]), int(date[6:])
    hour, min, sec = int(hours[0]), int(hours[1]), int(hours[2])
    dt = datetime(year,mon,day,hour, min,sec)
    
    # Assign coords
    f = f.assign_coords({'time2': dt})
    f = f.expand_dims(['time2'])
    f = f.drop_vars(['date', 'datesec'])
    f = f.squeeze(['time'])
    f = f.rename({'time2':'time'})

    return f

name =  'exp9init'
path = '/work/csp/mg20022/charts/CESM-DART/data/experiments'

files = glob.glob(f'{path}/{name}/{name}-*/{name}.dart.e.cam_forecast_mean.*.nc')

if len(files) > 0:

    f = xr.open_mfdataset(files, preprocess=preprocess)

In [11]:
# f.isel(time=0,lev=0)['VS'].where
posi = f['VS'].where(f['VS'] > 0.00001, np.nan).copy()
nega = f['VS'].where(f['VS'] <= 0.0001, np.nan).copy()

In [ ]:
sns.displot(x=posi.mean(dim=['lat']).values.ravel(), color='blue', kde=True,stat="density" )
sns.displot(x=nega.mean(dim=['lat']).values.ravel(), color='red', kde=True, stat="density" )

In [12]:
sns.kdeplot(x=posi.mean(dim=['lat']).values.ravel(), fill=True, label='Azul')
